## Import Libraries

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.metric_utils import rmse_score
import pandas as pd
import numpy as np
import pickle

## Load Input Data

In [2]:
with open(f'../Data/meta/catboost_single_predict_train_y.pickle', 'rb') as f:
    catboost_single_predict_train_y = pickle.load(f)

In [3]:
with open(f'../Data/meta/catboost_all_predict_train_y.pickle', 'rb') as f:
    catboost_all_predict_train_y = pickle.load(f)

In [4]:
catboost_single_predict_train_y.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.473838,-1.354527,-1.377475,-1.665377,-1.168665,-2.243439
1,-2.683915,-1.163297,-1.473901,-1.710264,-1.063420,-2.377334
2,-2.683915,-1.163297,-1.473901,-1.710264,-1.063420,-2.377334


In [5]:
catboost_all_predict_train_y.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.511625,-1.118764,-1.543356,-1.616733,-1.122452,-2.400809
1,-2.697706,-1.067591,-1.606706,-1.611239,-1.127114,-2.479767
2,-2.697706,-1.067591,-1.606706,-1.611239,-1.127114,-2.479767


## Load ground true data

In [6]:
with open(f'../Data/meta/train_y_after_feature_engineering.pickle', 'rb') as f:
    train_y_after_fe = pickle.load(f)

In [7]:
# You need to use original values of target variable to evaluate RMSE
with open(f'../Data/meta/non_scaled_train_y_after_feature_engineering.pickle', 'rb') as f:
    non_scaled_train_y = pickle.load(f)

## Stacking

In [8]:
with open(f'../Model/target_col_list.pickle', 'rb' ) as f:
    col_y_li = pickle.load(f)

In [9]:
data_dic = {}
for col in col_y_li:
    data_dic[col] = pd.concat([catboost_single_predict_train_y[col], catboost_all_predict_train_y[col]], axis=1)

In [10]:
model_dic = {}
for col in col_y_li:
    lr_model = LinearRegression()
    lr_model.fit(data_dic[col], train_y_after_fe[col])
    model_dic[col] = lr_model

In [11]:
with open(f'../Model/stacking_model_dic.pickle', 'wb' ) as f:
    pickle.dump(model_dic, f)

In [12]:
array_li= [ model_dic[col].predict(data_dic[col]).reshape(-1,1) for col in col_y_li ]
all_res = np.concatenate(array_li, axis=1)

In [13]:
# Load y-scaler model
with open(f'../Model/standard_scaler_y.pickle', 'rb') as f:
    scaler_y = pickle.load(f)

In [14]:
predictions = scaler_y.inverse_transform(all_res)
targets = non_scaled_train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value      6.285572
sensor_point6_i_value      7.733133
sensor_point7_i_value     12.763048
sensor_point8_i_value     10.315191
sensor_point9_i_value      9.621170
sensor_point10_i_value     9.119610
dtype: float64
